In [32]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [21]:
df = pd.read_csv('./Data/TwitterDataset.csv',index_col=0)
df.head()

,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,lang,location,profile_background_image_url,profile_image_url,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days,account_type
0,15/10/16 21:32,False,False,"Blame @xaiax, Inspired by @MakingInvisible, us...",4,1589,4,False,7.874060e+17,en,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/7874121826...,best_in_dumbest,11041,False,7.870,1403,bot
1,09/11/16 5:01,False,False,Photographing the American West since 1980. I ...,536,860,880,False,7.962160e+17,en,Estados Unidos,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/8023296328...,CJRubinPhoto,252,False,0.183,1379,human
2,17/06/17 5:34,False,False,Scruffy looking nerf herder and @twitch broadc...,3307,172,594,True,8.759500e+17,en,"Los Angeles, CA",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1278890453...,SVGEGENT,1001,False,0.864,1159,human
3,21/07/16 13:32,True,False,Wife.Godmother.Friend.Feline Fanatic! Assistan...,8433,517,633,True,7.561200e+17,en,"Birmingham, AL",NaN,http://pbs.twimg.com/profile_images/1284884924...,TinkerVHELPK5,1324,False,0.889,1489,human
4,15/01/12 16:32,False,False,Loan coach at @mancity & Aspiring DJ,88,753678,116,True,4.647813e+08,en,"England, United Kingdom",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9952566258...,JoleonLescott,4202,True,1.339,3138,human



# Dropping useless columns

In [26]:
df.drop('created_at', inplace= True, axis=1)
df.drop('profile_image_url', inplace= True, axis=1)
df.drop('profile_background_image_url', inplace= True, axis=1)
df.drop('id', inplace= True, axis=1)
df.head()

,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,lang,location,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days,account_type
0,False,False,"Blame @xaiax, Inspired by @MakingInvisible, us...",4,1589,4,False,en,unknown,best_in_dumbest,11041,False,7.870,1403,bot
1,False,False,Photographing the American West since 1980. I ...,536,860,880,False,en,Estados Unidos,CJRubinPhoto,252,False,0.183,1379,human
2,False,False,Scruffy looking nerf herder and @twitch broadc...,3307,172,594,True,en,"Los Angeles, CA",SVGEGENT,1001,False,0.864,1159,human
3,True,False,Wife.Godmother.Friend.Feline Fanatic! Assistan...,8433,517,633,True,en,"Birmingham, AL",TinkerVHELPK5,1324,False,0.889,1489,human
4,False,False,Loan coach at @mancity & Aspiring DJ,88,753678,116,True,en,"England, United Kingdom",JoleonLescott,4202,True,1.339,3138,human


In [34]:
#Label Encoding language column
df['lang'] = LabelEncoder().fit_transform(df['lang'])
df.head()

,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,lang,location,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days,account_type
0,False,False,"Blame @xaiax, Inspired by @MakingInvisible, us...",4,1589,4,False,9,unknown,best_in_dumbest,11041,False,7.870,1403,bot
1,False,False,Photographing the American West since 1980. I ...,536,860,880,False,9,Estados Unidos,CJRubinPhoto,252,False,0.183,1379,human
2,False,False,Scruffy looking nerf herder and @twitch broadc...,3307,172,594,True,9,"Los Angeles, CA",SVGEGENT,1001,False,0.864,1159,human
3,True,False,Wife.Godmother.Friend.Feline Fanatic! Assistan...,8433,517,633,True,9,"Birmingham, AL",TinkerVHELPK5,1324,False,0.889,1489,human
4,False,False,Loan coach at @mancity & Aspiring DJ,88,753678,116,True,9,"England, United Kingdom",JoleonLescott,4202,True,1.339,3138,human
